In [1]:
import pandas as pd
import pymc3 as pm
import numpy as np

C:\Users\zht702\Anaconda3\envs\pymc3_env\lib\site-packages\theano\scalar\basic.py:2412: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  self.ctor = getattr(np, o_type.dtype)


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
#from PIL import Image

In [ ]:
# Import the original dataset

df = pd.read_csv("data/sbl_isaiah_cmpls.csv").fillna("")

In [ ]:
# Define the contingency table

contingency_table_anim = pd.crosstab(df['construction'], df['cmpl_anim'])
contingency_table_anim

In [ ]:
df = contingency_table_anim

#### Establishing priors

Priors = our beliefs about the parameters before seeing the data
Priors are combined with the likelihood of the observed data

##### Different types of priors
- Uniform priors: I assume no prior knowledge, equal probability to all outcomes
- Informative Priors: Based on previous knowledge or literature, assign more weight to certain outcomes
- Weakly Informative Priors: assign broad probability, allow prior knowledge with a limited impact

##### Define the necessary probabilities, here for informative priors (important impact of literature):
- Probability of a complement being prepositional: P(prep) = 0.68
    - Medill, 2020, p.41, Strategies for Goal Markings:
        - 3125 observations, 2125 prep (68%)
- Probability of a complement being animate: P(anim) = 0.295
    - Medill, 2020, p.174, Goal-Marking Strategies by Goal Animacy
        - 3125 observations, 923 animate goals (29.5%)

In [ ]:
# I use the value from the contingency table
observed = df.values

In [ ]:
# Establishing the priors for the model

alpha_preps = np.array([6.8, 3.2]) # P(prep) = 0.68, P(non-prep) 0 0.32
alpha_anim = np.array([2.95, 7.05]) # P(anim) = 0.295, P(inanim) = 0.705

In [ ]:
# Model
with pm.Model() as model:
    # Dirichlet priors for the two categorical variables
    probs_preps = pm.Dirichlet('probs_preps', a=alpha_preps)
    probs_anim = pm.Dirichlet('probs_anim', a=alpha_anim)
    
    # Combining the probabilities to form a joint distribution
    # Using the outer product to get joint probabilities
    probs = pm.Deterministic('probs', tt.outer(probs_preps, probs_anim).ravel())
    
    # Reshape to match observed data dimensions
    probs = probs.reshape(observed.shape)
    
    # Expected counts
    n = observed.sum(axis=1)[:, None]
    expected = n * probs
    
    # Likelihood (sampling distribution) of observed data
    observed_data = pm.Multinomial('observed_data', n=n, p=probs, observed=observed)
    
    # Sample from the posterior
    trace = pm.sample(1000, tune=1000, return_inferencedata=True)

In [ ]:
import pymc3 as pm
import numpy as np
import theano.tensor as tt

# Sample Data
data = {
    'Category A': [10, 20, 30],
    'Category B': [20, 30, 40]
}

df = pd.DataFrame(data, index=['Group 1', 'Group 2', 'Group 3'])
observed = df.values

# Establishing priors based on given probabilities
alpha_preps = np.array([6.8, 3.2])  # P(prep) = 0.68, P(non-prep) = 0.32
alpha_anim = np.array([2.95, 7.05]) # P(anim) = 0.295, P(inanim) = 0.705

# Model
with pm.Model() as model:
    # Dirichlet priors for the two categorical variables
    probs_preps = pm.Dirichlet('probs_preps', a=alpha_preps)
    probs_anim = pm.Dirichlet('probs_anim', a=alpha_anim)
    
    # Combining the probabilities to form a joint distribution
    probs = pm.math.outer(probs_preps, probs_anim)
    
    # Reshape to match observed data dimensions
    probs = probs.reshape(observed.shape)
    
    # Expected counts
    n = observed.sum(axis=1)[:, None]
    expected = n * probs
    
    # Likelihood (sampling distribution) of observed data
    observed_data = pm.Multinomial('observed_data', n=n, p=probs, observed=observed)
    
    # Sample from the posterior
    trace = pm.sample(1000, tune=1000, return_inferencedata=True)

# Display results
import arviz as az
az.plot_trace(trace)